# 利用最小二乘法计算贵州茅台和宁德时代的β和α值

In [1]:
import pandas as pd
import tushare as ts
import statsmodels.api as sm

# 初始化（替换为你的token）
ts.set_token('2d522f171c5a992e9a84aceb6e6aa90489cb9ef60a453d1ed122e1d7')
pro = ts.pro_api()

# 获取贵州茅台数据
stock_prices = pro.daily(ts_code='600519.SH', start_date='20170101', end_date='20230101')
stock_prices.set_index('trade_date', inplace=True)
stock_prices.index = pd.to_datetime(stock_prices.index)
stock_prices.sort_index(inplace=True)

#获取宁德时代数据
ningde_prices = pro.daily(ts_code ='300750.SZ',start_date='20170101',end_date='20230101')
ningde_prices.set_index('trade_date',inplace=True)
ningde_prices.index =pd.to_datetime(ningde_prices.index)
ningde_prices.sort_index(inplace=True)


# 获取沪深300指数数据
market_prices = pro.index_daily(ts_code='000300.SH', start_date='20170101', end_date='20230101')
market_prices.set_index('trade_date', inplace=True)
market_prices.index = pd.to_datetime(market_prices.index)
market_prices.sort_index(inplace=True)

# 获取中国10年期国债收益率数据
bond_yield = pro.yc_cb(ts_code='1001.CB',curve_type='1',curve_term='10',start_date='20170101', end_date='20230101' )
bond_yield.set_index('trade_date', inplace=True)
bond_yield.index = pd.to_datetime(bond_yield.index)
bond_yield.sort_index(inplace=True)

# 计算股票和市场的日收益率
stock_returns = stock_prices['close'].pct_change().dropna()
ningde_returns=ningde_prices['close'].pct_change().dropna()
market_returns = market_prices['close'].pct_change().dropna()

# 计算每日无风险利率（假设年化收益率）
daily_risk_free_rate = bond_yield['yield'] / 100 / 242

# 对齐日期索引
df = pd.concat([stock_returns,  market_returns, daily_risk_free_rate], axis=1, join='inner',keys=['stock','market', 'risk_free']).dropna()
df1=pd.concat([ningde_returns,market_returns,daily_risk_free_rate],axis=1,join='inner',keys=['ningde','market','risk_free']).dropna()

# 计算超额收益
excess_stock_returns = df['stock'] - df['risk_free']
excess_ningde_returns=df1['ningde']- df1['risk_free']
excess_market_returns1 = df['market'] - df['risk_free']
excess_market_returns2 = df1['market'] - df1['risk_free']

# 添加常数项
excess_market_returns1 = sm.add_constant(excess_market_returns1)
excess_market_returns2=sm.add_constant(excess_market_returns2)

# 最小二乘法回归
model1 = sm.OLS(excess_stock_returns, excess_market_returns1).fit()
model2 =sm.OLS(excess_ningde_returns,excess_market_returns2).fit()

# 打印回归结果
print("\033[34m贵州茅台最小二乘法结果\033[0m：")
print('\n')
print(model1.summary())
print('\n')
print("\033[31m宁德时代最小二乘法结果\033[0m：")  
print('\n')
print(model2.summary())

贵州茅台最小二乘法结果：


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.437
Model:                            OLS   Adj. R-squared:                  0.437
Method:                 Least Squares   F-statistic:                     1132.
Date:                Wed, 19 Mar 2025   Prob (F-statistic):          4.34e-184
Time:                        14:31:10   Log-Likelihood:                 4046.9
No. Observations:                1458   AIC:                            -8090.
Df Residuals:                    1456   BIC:                            -8079.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0011      0.000     

# 利用协方差法计算宁德时代和贵州茅台β值

In [3]:
# 导入必要库
import tushare as ts
import numpy as np
import pandas as pd
import statsmodels.api as sm

# 获取历史数据（示例使用贵州茅台和沪深300指数）
ts.set_token('2d522f171c5a992e9a84aceb6e6aa90489cb9ef60a453d1ed122e1d7')
pro = ts.pro_api()

stock_prices1 = pro.daily(ts_code='600519.SH',start_date='20130101',end_date='20230101')['close']
stock_prices2 =pro.daily(ts_code='300750.SZ',start_date='20130101',end_date='20230101')['close']
market_prices =pro.index_daily(ts_code='000300.SH',start_date='20130101',end_date='20230101')['close']

# 计算日收益率（百分比变化）
returns = pd.DataFrame({
    'maotai_stock': stock_prices1.pct_change().dropna(),
    'ningde_stock':stock_prices2.pct_change().dropna(),
    'market': market_prices.pct_change().dropna()
}).dropna()

# 方法1：协方差法计算β
cov_matrix1 = np.cov(returns['maotai_stock'], returns['market'])
cov_matrix2=np.cov(returns['ningde_stock'],returns['market'])
beta_cov1 = cov_matrix1[0, 1] / cov_matrix1[1, 1]
beta_cov2=cov_matrix2[0,1]/cov_matrix2[1,1]


# 结果输出
print(f"贵州茅台β值: {beta_cov1:.4f}")
print(f"宁德时代β值:{beta_cov2:.4f}")

贵州茅台β值: 1.0932
宁德时代β值:1.2565
